# Get Soup

In [52]:
from bs4 import BeautifulSoup
import requests
import regex as re
import pandas as pd

In [53]:
page = requests.get("https://www.laughingplace.com/w/p/disneyland-current-wait-times/")
dland = BeautifulSoup(page.content, 'html.parser')

In [54]:
page2 = requests.get("https://www.laughingplace.com/w/p/disney-california-adventure-current-wait-times/")
dca = BeautifulSoup(page2.content, 'html.parser')

## Get Attractions Table

In [71]:
waits = []

In [72]:
check = dland.find('div', class_='header').text
check

'Operating Hours For Fri, Oct 8: 8a-11p\n \nLast Check at 3:37pm on Friday'

In [73]:
check = check.split('\n')

In [74]:
pattern = 'Operating Hours For '
date_string = re.sub(pattern, '', check[0])
date_string = date_string[5:11]
date = re.sub('[^A-Za-z0-9]+', ' ', date_string).strip()
date

'Oct 8'

In [75]:
pattern = 'Last Check at '
mod_string = re.sub(pattern, '', check[2])
mod_string = mod_string[:6]
time = re.sub('[^A-Za-z0-9]+', '', mod_string).strip()
time

'337pm'

In [76]:
hours_string = check[0][31:]
hours_string=hours_string.strip()
hours_string

'8a-11p'

In [77]:
total_date=date+' '+hours_string+' Pull:'+time
total_date

'Oct 8 8a-11p Pull:337pm'

# Disneyland

## Wait Times

In [78]:
first = dland.find('table', class_='lp_attraction')

In [79]:
wait_times = first.find_all("span", id=re.compile("f"))

In [80]:
for item in wait_times:
    
    if item.find('div', class_='waiticon'):
        raw_time = item.find('td')
        time = raw_time.text[:3].strip(' ')
        if ' ' in time:
            time = time[:1]
    else:
        time = item.text[:3].strip(' ')
        if ' ' in time:
            time = time[:1]
        
    waits.append(time)

# DCA

## Wait Times

In [81]:
first = dca.find('table', class_='lp_attraction')

In [82]:
wait_times = first.find_all("span", id=re.compile("f"))
wait_times

[<span id="f_the_bakery_tour">Closed</span>,
 <span id="f_carthay_circle_restaurant">Open</span>,
 <span id="f_golden_zephyr"><span class="statusdown">Down</span></span>,
 <span id="f_goofys_sky_school">35 minutes</span>,
 <span id="f_grizzly_river_run">35 minutes</span>,
 <span id="f_guardians_of_the_galaxy__mission_breakout">Closed</span>,
 <span id="f_incredicoaster">30 minutes</span>,
 <span id="f_inside_out_emotional_whirlwind">20 minutes</span>,
 <span id="f_jessies_critter_carousel">5 minutes</span>,
 <span id="f_jumpin_jellyfish">10 minutes</span>,
 <span id="f_lamplight_lounge___boardwalk_dining">45 minutes</span>,
 <span id="f_the_little_mermaid__ariels_undersea_adventure">15 minutes</span>,
 <span id="f_magic_key_terrace_walk_up_only">Open</span>,
 <span id="f_mickeys_philharmagic"><span class="statusdown">Down</span></span>,
 <span id="f_monsters_inc_mike__sulley_to_the_rescue">
 <div class="waiticon">
 <table>
 <tr>
 <td>30 minutes</td>
 <td><img sizes="(max-width: 37px) 1

In [83]:
for item in wait_times:
   
    if item.find('div', class_='waiticon'):
        raw_time = item.find('td')
        time = raw_time.text[:3].strip(' ')
        if ' ' in time:
            time = time[:1]
    else:
        time = item.text[:3].strip(' ')
        if ' ' in time:
            time = time[:1]
        
    waits.append(time)

## DataFrame

In [84]:
waits

['35',
 '35',
 '30',
 'Ref',
 '30',
 '10',
 'Clo',
 '10',
 'Clo',
 '35',
 'Ope',
 'Clo',
 '35',
 'Clo',
 'Clo',
 'Dow',
 '50',
 '30',
 '50',
 '5',
 '15',
 'Clo',
 '5',
 'Ope',
 '40',
 'Clo',
 '40',
 'Clo',
 '20',
 'Ope',
 '35',
 '15',
 '5',
 '25',
 '30',
 'Ref',
 'Clo',
 '25',
 '60',
 '50',
 '20',
 'Clo',
 '30',
 'Ref',
 'Ope',
 'Clo',
 'Ope',
 'Dow',
 '35',
 '35',
 'Clo',
 '30',
 '20',
 '5',
 '10',
 '45',
 '15',
 'Ope',
 'Dow',
 '30',
 '45',
 '20',
 '100',
 'Clo',
 'Ope',
 '20',
 '35',
 '50',
 'Dow']

In [85]:
attraction_waits = pd.read_pickle('attraction_waits.pkl')
attraction_waits

,Oct 8 8a-11p @325pm
Alice in Wonderland,25
Astro Orbitor,35
Autopia,40
Big Thunder Mountain Railroad,Ref
Buzz Lightyear Astro Blasters,25
...,...
Redwood Creek Challenge Trail,Ope
Silly Symphony Swings,20
Soarin’ Around the World,35
Toy Story Midway Mania!,45


In [86]:
attraction_waits[total_date] = waits
attraction_waits

,Oct 8 8a-11p @325pm,Oct 8 8a-11p Pull:337pm
Alice in Wonderland,25,35
Astro Orbitor,35,35
Autopia,40,30
Big Thunder Mountain Railroad,Ref,Ref
Buzz Lightyear Astro Blasters,25,30
...,...,...
Redwood Creek Challenge Trail,Ope,Ope
Silly Symphony Swings,20,20
Soarin’ Around the World,35,35
Toy Story Midway Mania!,45,50


In [87]:
attraction_waits.to_pickle('attraction_waits.pkl')